# Sympy experiments


# Sympy playground

In [ ]:
import sympy
sympy.init_printing()
import sympy.printing.c as ccode
import sympy.printing.c as codegen

from IPython.display import display, Markdown
from sympy.functions import exp
from sympy import diff
from IPython.display import Latex

## tansig

In [ ]:
phi,z = sympy.symbols(r'\phi z')
phi = 2 / (1 + exp(-2 * z)) - 1
phi

In [ ]:
sympy.simplify(diff(phi))

In [ ]:
sympy.simplify(0.5 * (1 - (phi**2)))

## Biquad transformation 

In [ ]:
s, m,c,d,n,a,b,z,y, T = sympy.symbols("s m c d n a b z y T")

Biquad transfer function

In [ ]:
Hn = m*s**2 + c*s + d
Hd = n*s**2 + a*s + b
H = Hn / Hd
H

Substitute 
$s\rightarrow \frac{2}{T}\cdot\frac{z-1}{z+1}$

In [ ]:
H.subs(s,(2/T)*(z-1)/(z+1))

In [ ]:
sympy.collect(sympy.simplify(sympy.collect(sympy.expand(H.subs(s,(2/T)*(z-1)/(z+1))),z)),z)

rewrite in terms of $z^{-1}$ delay notation and refactoring the time to get it to the format used in SimulC:

$$\frac{
\left( d - 2 c/T + 4 m/T^{2}\right)z^{-2} + 
 \left(2 d - 8 m/ T^{2}\right)z^{-1} + 
\left( d + 2  c/T + 4 m/T^{2}\right) 
}{
\left( b - 2 a/T + 4 n/T^{2}\right)z^{-2} + 
 \left(2  b - 8 n/T^{2}\right)z^{-1} +
\left( b + 2 a/T + 4 n/T^{2}\right) 
}
$$

## FEM solution

In [ ]:
a, b, x, y, x1, x2, x3, x4, y1, y2, y3, y4 = sympy.symbols("a, b, x, y, x_1 x_2 x_3 x_4 y_1 y_2 y_3 y_4")

Solve the set of both equations for a and b


In [ ]:
sols = sympy.solve ([
    (1-a)*(1-b)*x1/4 + (1+a)*(1-b)*x2/4 + (1+a)*(1+b)*x3/4 + (1-a)*(1+b)*x4/4 - x , 
    (1-a)*(1-b)*y1/4 + (1+a)*(1-b)*y2/4 + (1+a)*(1+b)*y3/4 + (1-a)*(1+b)*y4/4 - y
    ] , 
    [a, b]
    )


In [ ]:
print(len(sols), len(sols[0]), len(sols[1]))


## a1

In [ ]:
sympy.simplify(sols[0][0])

## b1

In [ ]:
sympy.simplify(sols[0][1])

## a2

In [ ]:
sympy.simplify(sols[1][0])

## b2

In [ ]:
sympy.simplify(sols[1][1])

In [ ]:
print(sympy.latex(sols[1][0]))

Evaluate at specific point

In [ ]:
subsdic = {
    x1:1.1, y1:0.6,
    x2:2.9,y2:-1.4,
    x3:3.9,y3:3.4,
    x4:1.6,y4:3.4,
    x:3.34,y:0.01
}


In [ ]:
a1 = sols[0][0].subs(subsdic)
b1 = sols[0][1].subs(subsdic)
a2 = sols[1][0].subs(subsdic)
b2 = sols[1][1].subs(subsdic)
print(f'a1={a1}\nb1={b1}\na2={a2}\nb2={b2}')

In [ ]:
print(f'a1={ccode(sols[0][0])}')

In [ ]:
print(f'b1={ccode(sols[0][1])}')

In [ ]:
print(f'a2={ccode(sols[1][0])}')

In [ ]:
print(f'b2={ccode(sols[1][1])}')

# Deriving yaw-pitch-roll Euler angle equations

In [ ]:
# general setup
import sympy
sympy.init_printing()
from sympy.printing.mathml import print_mathml
from IPython.display import display

In [ ]:
# elementary symbols
phi, theta, psi = sympy.symbols(r'\phi \theta \psi')
wx, wy, wz = sympy.symbols(r'\omega_x \omega_y \omega_z')
t = sympy.symbols('t')

In [ ]:
# main variables of interest
def vec(x, y, z): # helper: make a vector
    return sympy.Matrix([[x], [y], [z]])

# angular rate vector
w = vec(wx, wy, wz)

# Euler rate "vector"
dpsi,dtheta,dphi = [sympy.diff(x,t,evaluate=False) for x in [psi,theta,phi]]
deib = vec(dphi, dtheta, dpsi)

In [ ]:
# rotation matrices
s, c = sympy.sin, sympy.cos

def Ephi(phi): return sympy.Matrix([[1,  0,      0],
                                    [0,  c(phi), s(phi)],
                                    [0, -s(phi), c(phi)]])
def Ethe(the): return sympy.Matrix([[c(the), 0, -s(the)],
                                    [0,      1,  0],
                                    [s(the), 0,  c(the)]])
def Epsi(psi): return sympy.Matrix([[ c(psi), s(psi), 0],
                                    [-s(psi), c(psi), 0],
                                    [0,       0,      1]])

In [ ]:
# intermediate frame 2 to body (roll rotation)
H2b = Ephi(phi)
# intermediate frame 1 to frame 1 (pitch rotation)
H12 = Ethe(theta)

# the core equation; each euler rate can be expressed as an angular rate about some axis;
# these sum to the total angular rate, but we must rotate everything to the body axes.
w_sum_deib = vec(dphi, 0, 0) + H2b * vec(0, dtheta, 0) + H2b * H12 * vec(0, 0, dpsi)

display(sympy.Equality(w, w_sum_deib))

In [ ]:
sol = sympy.solve(w - w_sum_deib, deib)
for var in deib:
    display(sympy.Equality(var, sol[var]))

## convert to python functions

    vignet =(np.cos(np.pi*vangl/(2*angles[-1])))**0.1 + 0.09

In [ ]:
import sympy
import numpy as np
from sympy.codegen.ast import Assignment

t, p,v, x,y,s, m,c,d,n,a,b,z,y, T = sympy.symbols("\\theta p v x y s m c d n a b z y T")

y = (sympy.cos(sympy.pi * a / (2. * m)))**p + c
mstr = sympy.latex(y)
subsdic = {c:0.09, p:0.1,m:0.471}
T = y.subs(subsdic)
lstr = sympy.latex(T)

print(mstr)
display(y)
display(T)

In [ ]:
# pstr = sympy.pycode(Assignment(v, T)).replace('math','np')
pstr = sympy.pycode(T).replace('math','np')

print(pstr)
a = np.linspace(0,0.471,100)
eval(pstr)

In [ ]:
import pyradi.ryplot as ryplot
lstr = lstr.replace(r' \pi a ',r' \cdot\pi\cdot \theta ')
print(lstr)
p = ryplot.Plotter(1,1,1,f'${lstr}$',(12,5))
p.plot(1,a,eval(pstr))
                   
                   

## Solve circle 

In [ ]:
# import sympy
# import numpy as np
# from sympy.codegen.ast import Assignment

y1,y2,yc,z1,z2,zc,r = sympy.symbols("y_1,y_2,y_c,z_1,z_2 z_c r")
eqzc = sympy.solve((y1-yc)**2 +(z1-zc)**2-((y2-yc)**2 +(z2-zc)**2),zc)
print('Equation for zc')
display(eqzc[0])
eqr = sympy.solve((y1-yc)**2 +(z1-zc)**2-r**2,r)
print('Equation for r')
display(eqr[1])

print('---------------------------------------')

dicsubs = {y1:1.2,y2:2.5,z1:1.5,z2:2.5,yc:1.1}
zcval = float(eqzc[0].subs(dicsubs))
print(f'z_c = {zcval}')
r = eqr[1].subs(dicsubs).subs({zc:zcval})
print(f'r = {r}')

print('---------------------------------------')
print(sympy.python(eqzc))
print(sympy.python(eqr))


# Quadrilateral and Area 

The objective is to solve the angles for a quadrilateral given the length of the four edges and the area of the polygon.
The process is to divide the quadrilateral into two triangles, writing the area of each of these triangles in terms of the diagonal, then solve for the diagonal length such that the sum of areas of the two triangles adds up to the total area.

Name the four edges of the quadrilateral as $a,b,c,d$ and the line connecting the opposite points as $z$

Heron's formula gives the area, $A$, of a triangle with sides $a, b, z$ as:
$$
A=\sqrt{s(s-a)(s-b)(s-z)} \text { where } s=\frac{1}{2}(a+b+z).
$$

The 



In [18]:
# O, W, S, N = sympy.symbols("O W S N")
def getangle(c,d,zi):
    O = sympy.acos(-(z**2 - c**2 - d**2)/(2*c*d))
    return np.double(180*O.subs([(c,c),(d,c)]).subs([(z,zi)])/np.pi)

def movev(xi,yi,dist,angle):
    xd = dist * np.cos(np.pi*angle/180)
    yd = dist * np.sin(np.pi*angle/180)
    return xi+xd,yi+yd


In [47]:
import sympy
import numpy as np
import pandas as pd


a, b, c, d, z, s1, s2, A1, A2, A = sympy.symbols("a b c d z s_1 s_2 A_1 A_2 A")
s1 = (a+b+z) / 2
eq1 = sympy.sqrt(s1 * (s1-a) * (s1-b) * (s1-z)) - A1
s2 = (c+d+z) / 2
eq2 = sympy.sqrt(s2 * (s2-c) * (s2-d) * (s2-z)) - A2

Az1 = sympy.solve(eq1,A1)
Az2 = sympy.solve(eq2,A2)

Atot = Az1[0]+ Az2[0] - A
display(Atot)

# display(sympy.solve(Atot,z)[-1])


-A + sqrt(-a**4 + 2*a**2*b**2 + 2*a**2*z**2 - b**4 + 2*b**2*z**2 - z**4)/4 + sqrt(-c**4 + 2*c**2*d**2 + 2*c**2*z**2 - d**4 + 2*d**2*z**2 - z**4)/4

In [136]:

dicV = {}

pd.set_option('display.chop_threshold', 0.001)

dicV['2003'] = {a:15.528,b:31.764,c:30.,d:25.246,A:591.7,'zl':39.532}

# dicV['CJW'] = {a:15.5,b:31.47,c:30,d:25,A:608,'zl':38.9725963248928}
# dicV['GIS'] = {a:15.5,b:31.622,c:29.9,d:24.993,A:607.68,'zl':39.08}

# 38.1580113680703


for key in dicV.keys():

    # solve for diagonal, it takes a while
    if dicV[key]['zl'] is None:
        Atot = Atot.subs(dicV[key])
        print(f'{Atot}')
        zl = sympy.solve(Atot,z)[-1]
        display(zl)
        dicV[key]['zl'] = zl 

    dicA = {'Wes  ':{'s':1},'Oos  ':{'s':3},'Noord':{'s':2},'Suid ':{'s':0},}
    dicA['Wes  ']['A'] = getangle(dicV[key][a],dicV[key][b],dicV[key]['zl'])
    dicA['Oos  ']['A'] = getangle(dicV[key][c],dicV[key][d],dicV[key]['zl'])
    dicA['Noord']['A'] = getangle(dicV[key][b],dicV[key]['zl'],dicV[key][a]) +  getangle(dicV[key][d],dicV[key]['zl'],dicV[key][c])
    dicA['Suid ']['A'] = getangle(dicV[key][a],dicV[key]['zl'],dicV[key][b]) +  getangle(dicV[key][c],dicV[key]['zl'],dicV[key][d])
    if np.fabs(dicA['Wes  ']['A']+dicA['Oos  ']['A']+dicA['Noord']['A']+dicA['Suid ']['A'] - 360) > 0.1:
        print('Check angles')
    
    # initial angle from S to O, in deg
    sA = 35.086
    if False:
        x = 10.327
        y = 11.081
    else:
        x = 0
        y = 0   
    
    vpath = [[dicV[key][c],sA,'Oos  '],
             [dicV[key][d],sA+180-dicA['Oos  ']['A'],'Noord'],
             [dicV[key][b],sA+360-dicA['Oos  ']['A']-dicA['Noord']['A'],'Wes  '],
             [dicV[key][a],sA+540-dicA['Oos  ']['A']-dicA['Noord']['A']-dicA['Wes  ']['A'],'Suid ']]
    
    for iseg,segment in enumerate(vpath):
        # segment  = vpath[iseg%len(vpath)]
        x,y = movev(x,y,segment[0],segment[1])
        dicA[segment[2]]["x"],dicA[segment[2]]["y"] = x,y
        dicA[segment[2]]["L"] = 0

    dfCorners = pd.DataFrame.from_dict(dicA).T
    dfCorners = dfCorners.sort_values(by='s')
    dfCorners.at['Suid ','L'] = legLength(dfCorners.loc[['Suid ']],dfCorners.loc[['Wes  ']])
    dfCorners.at['Wes  ','L'] = legLength(dfCorners.loc[['Wes  ']],dfCorners.loc[['Noord']])
    dfCorners.at['Noord','L'] = legLength(dfCorners.loc[['Noord']],dfCorners.loc[['Oos  ']])
    dfCorners.at['Oos  ','L'] = legLength(dfCorners.loc[['Oos  ']],dfCorners.loc[['Suid ']])
    area = dfCorners.loc[['Wes  ']]['L'][0] * dfCorners.loc[['Noord']]['L'][0]*np.sin(np.pi*dfCorners.loc[['Noord']]['A'][0]/180)
    area += dfCorners.loc[['Oos  ']]['L'][0] * dfCorners.loc[['Suid ']]['L'][0]*np.sin(np.pi*dfCorners.loc[['Suid ']]['A'][0]/180)
    area = area / 2
                                                                          
    print(f'{key} model: \nArea = {area:.2f}')
    # print(dicV[key])                                                                                         
    display(dfCorners.drop(['s'],axis=1).style.format({'A':'{:.1f}','x':'{:.3f}','y':'{:.3f}','L':'{:.2f}'}))
    # print('-----------------------------')

    
#     constants = pd.DataFrame([('pi',np.pi),('e',np.e)],
#                    columns=['name','value'])
# C = constants.style.format({'name': '~~ {} ~~', 'value':'--> {:15.10f} <--'})
# C
    

2003 model: 
Area = 612.53


,A,x,y,L
Suid,89.3,-0.000,0.000,15.53
Wes,108.5,-8.776,12.810,31.76
Noord,71.2,10.386,38.143,25.25
Oos,91.0,24.549,17.244,30.00


In [92]:
def legLength(ori,end):
    len = np.sqrt((end.iloc[[0]]['x'][0]-ori.iloc[[0]]['x'][0])**2+(end.iloc[[0]]['y'][0]-ori.iloc[[0]]['y'][0])**2)
    return len